# 5. Composição de Múltiplas Vistas

Ao visualizar uma quantidade de campos de dados diferentes, podemos ser tentados a usar o maior número possível de canais de codificação visual: `x`, `y`, `color`, `size`, `shape` e assim por diante. No entanto, à medida que o número de canais de codificação aumenta, um gráfico pode rapidamente tornar-se confuso e difícil de ler e de ser compreendido. Desta forma, uma alternativa para “sobrecarregar” um único gráfico é _compor vários gráficos_ de uma forma que facilite comparações rápidas.

Neste capítulo, examinaremos uma variedade de operações para a _composição de múltiplas vistas_ :

- _camada_  (`layer`): coloca gráficos compatíveis diretamente uns sobre os outros,
- _faceta_ (`facet`): divide os dados em vários gráficos, organizados em linhas ou colunas,
- _concatenação_  (`concatenate`): posiciona gráficos arbitrários dentro de um layout (esquema) partilhado, e
- _repetição_ (`repeat): pega uma especificação de gráfico de base e aplica a vários campos de dados.

Veremos então como essas operações formam uma _álgebra de composição de visualizações_, na qual as operações podem ser combinadas para construir uma variedade de exibições complexas de múltiplas visualizações.

_Este capítulo faz parte do [currículo de visualização de dados](https://github.com/uwdata/visualization-curriculum)._

In [ ]:
import pandas as pd
import altair as alt

## 5.1. Dados meteorológicos

Inicialmente, vamos visualizar estatísticas meteorológicas para as cidades americanas de Seattle e Nova Iorque. Vamos carregar o conjunto de dados e observar as primeiras e as últimas 10 linhas:

In [ ]:
clima = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/weather.csv'

In [ ]:
df = pd.read_csv(clima)
df.head(10)

,location,date,precipitation,temp_max,temp_min,wind,weather
0,Seattle,2012-01-01,0.0,12.8,5.0,4.7,drizzle
1,Seattle,2012-01-02,10.9,10.6,2.8,4.5,rain
2,Seattle,2012-01-03,0.8,11.7,7.2,2.3,rain
3,Seattle,2012-01-04,20.3,12.2,5.6,4.7,rain
4,Seattle,2012-01-05,1.3,8.9,2.8,6.1,rain
5,Seattle,2012-01-06,2.5,4.4,2.2,2.2,rain
6,Seattle,2012-01-07,0.0,7.2,2.8,2.3,rain
7,Seattle,2012-01-08,0.0,10.0,2.8,2.0,sun
8,Seattle,2012-01-09,4.3,9.4,5.0,3.4,rain
9,Seattle,2012-01-10,1.0,6.1,0.6,3.4,rain


In [ ]:
df.tail(10)

,location,date,precipitation,temp_max,temp_min,wind,weather
2912,New York,2015-12-22,4.8,15.6,11.1,3.8,fog
2913,New York,2015-12-23,29.5,17.2,8.9,4.5,fog
2914,New York,2015-12-24,0.5,20.6,13.9,4.9,fog
2915,New York,2015-12-25,2.5,17.8,11.1,0.9,fog
2916,New York,2015-12-26,0.3,15.6,9.4,4.8,drizzle
2917,New York,2015-12-27,2.0,17.2,8.9,5.5,fog
2918,New York,2015-12-28,1.3,8.9,1.7,6.3,snow
2919,New York,2015-12-29,16.8,9.4,1.1,5.3,fog
2920,New York,2015-12-30,9.4,10.6,5.0,3.0,fog
2921,New York,2015-12-31,1.5,11.1,6.1,5.5,fog


Agora, criaremos multivisualizações para examinar o clima dentro e fora das cidades.

## 5.2. Camada

Uma das formas mais comuns de combinar vários gráficos é colocar marcas umas em cima das outras. Se os domínios de escala subjacentes forem compatíveis, podemos juntá-los para formar _eixos partilhados_. Se uma das codificações `x` ou `y` não for compatível, podemos criar um _gráfico de eixo duplo_, que sobrepõe marcas usando escalas e eixos separados.

### 5.2.1. Eixos Partilhados

Vamos começar traçando as temperaturas médias mínimas e máximas por mês:

In [19]:
alt.Chart(clima).mark_area().encode(
  alt.X('month(date):T', title = 'Meses'),
  alt.Y('average(temp_max):Q', title = 'Média da temp_max'),
  alt.Y2('average(temp_min):Q', title = 'Média da temp_min')
)

alt.Chart(...)

_O gráfico acima mostra os intervalos de temperatura para cada mês ao longo da totalidade dos nossos dados. No entanto, isto é bastante enganador, uma vez que agrega as medições de Seattle e Nova Iorque!_

Uma solução é subdividir os dados por localização utilizando uma codificação de cor, ao mesmo tempo que ajustamos a opacidade da marca para acomodar áreas sobrepostas:

In [21]:
alt.Chart(clima).mark_area(opacity=0.3).encode(
  alt.X('month(date):T', title = 'Meses'),
  alt.Y('average(temp_max):Q', title = 'Média da temp_max'),
  alt.Y2('average(temp_min):Q', title = 'Média da temp_min'),
  alt.Color('location:N', title = 'Localização')
)

alt.Chart(...)

_Desta forma, podemos ver que Seattle é mais temperada: mais quente no inverno e mais fresca no verão._

Neste caso, criamos um gráfico em camadas sem quaisquer recursos especiais simplesmente subdividindo as marcas de área por cor. Enquanto o gráfico acima nos mostra as faixas de temperatura, também podemos querer enfatizar o meio da faixa.

Vamos criar um gráfico de linhas mostrando o ponto médio da temperatura média. Utilizaremos uma transformação `calculate` para calcular os pontos médios entre as temperaturas mínimas e máximas diárias:

In [22]:
alt.Chart(clima).mark_line().transform_calculate(
  temp_mid='(+datum.temp_min + +datum.temp_max) / 2'
).encode(
  alt.X('month(date):T', title = 'Meses'),
  alt.Y('average(temp_mid):Q', title = 'Temperatura média'),
  alt.Color('location:N', title = 'Localização')
)

alt.Chart(...)

_Nota_ : observe a utilização de `+datum.temp_min` na transformação de cálculo. Como estamos carregando os dados diretamente de um arquivo CSV sem nenhuma instrução especial de análise, os valores de temperatura podem ser representados internamente como valores de string. Adicionar o `+` na frente do valor força-o a ser tratado como um número.

Agora, podemos combinar esses gráficos colocando as linhas de ponto médio sobre as áreas de intervalo. Usando a sintaxe `chart1 + chart2`, podemos especificar que queremos um novo gráfico em camadas no qual `chart1` é a primeira camada e `chart2` é uma segunda camada desenhada em cima:

In [23]:
tempMinMax = alt.Chart(clima).mark_area(opacity=0.3).encode(
  alt.X('month(date):T', title = 'Meses'),
  alt.Y('average(temp_max):Q', title = 'Média da temp_max'),
  alt.Y2('average(temp_min):Q', title = 'Média da temp_min'),
  alt.Color('location:N', title = 'Localização')
)

tempMid = alt.Chart(clima).mark_line().transform_calculate(
  temp_mid='(+datum.temp_min + +datum.temp_max) / 2'
).encode(
  alt.X('month(date):T', title = 'Meses'),
  alt.Y('average(temp_mid):Q', title = 'Temperatura média'),
  alt.Color('location:N', title = 'Localização')
)

tempMinMax + tempMid

alt.LayerChart(...)

_Agora temos um gráfico de várias camadas! No entanto, o título do eixo y (embora informativo) tornou-se um pouco longo e indisciplinado..._

Vamos personalizar os nossos eixos para limpar o gráfico. Se definirmos um título de eixo personalizado numa das camadas, este será automaticamente utilizado como um título de eixo partilhado para todas as camadas:

In [25]:
tempMinMax = alt.Chart(clima).mark_area(opacity=0.3).encode(
  alt.X('month(date):T', title=None, axis=alt.Axis(format='%b')),
  alt.Y('average(temp_max):Q', title='Temperatura Média °C'),
  alt.Y2('average(temp_min):Q'),
  alt.Color('location:N', title = 'Localização')
)

tempMid = alt.Chart(clima).mark_line().transform_calculate(
  temp_mid='(+datum.temp_min + +datum.temp_max) / 2'
).encode(
  alt.X('month(date):T'),
  alt.Y('average(temp_mid):Q'),
  alt.Color('location:N')
)

tempMinMax + tempMid

alt.LayerChart(...)

_O que acontece se ambas as camadas tiverem títulos de eixo personalizados? Modifique o código acima para descobrir..._

Acima utilizamos o operador `+`, uma abreviatura conveniente para o método `layer` do Altair. Podemos gerar um gráfico de camadas idêntico utilizando diretamente o método `layer`:

In [26]:
alt.layer(tempMinMax, tempMid)

alt.LayerChart(...)

Note que a ordem das entradas para uma camada é importante, uma vez que as camadas subsequentes serão desenhadas em cima das camadas anteriores. _Tente trocar a ordem dos gráficos nas células acima. O que acontece? (Dica: observe atentamente a cor das marcas de `linha`)._

### 5.2.2. Gráficos de Eixo Duplo

_Seattle tem a reputação de ser uma cidade chuvosa. Será que isso é merecido?_

Para responder a pergunta, vamos analisar a precipitação juntamente com a temperatura para saber mais. Primeiro, vamos criar um gráfico de base que mostra a precipitação média mensal em Seattle:

In [27]:
alt.Chart(clima).transform_filter(
  'datum.location == "Seattle"'
).mark_line(
  interpolate='monotone',
  stroke='grey'
).encode(
  alt.X('month(date):T', title=None),
  alt.Y('average(precipitation):Q', title='Precipitação')
)

alt.Chart(...)

Para facilitar a comparação com os dados de temperatura, vamos criar um novo gráfico em camadas. Aqui está o que acontece se tentarmos colocar os gráficos em camadas como fizemos anteriormente:

In [29]:
tempMinMax = alt.Chart(clima).transform_filter(
  'datum.location == "Seattle"'
).mark_area(opacity=0.3).encode(
  alt.X('month(date):T', title=None, axis=alt.Axis(format='%b')),
  alt.Y('average(temp_max):Q', title='Temperatura Média °C'),
  alt.Y2('average(temp_min):Q')
)

precip = alt.Chart(clima).transform_filter(
  'datum.location == "Seattle"'
).mark_line(
  interpolate='monotone',
  stroke='grey'
).encode(
  alt.X('month(date):T'),
  alt.Y('average(precipitation):Q', title='Precipitação')
)

alt.layer(tempMinMax, precip)

alt.LayerChart(...)

_Os valores de precipitação utilizam uma faixa muito mais pequena do eixo y do que as temperaturas!_

Por padrão, os gráficos em camadas usam um domínio compartilhado: os valores para o eixo x ou eixo y são combinados em todas as camadas para determinar uma extensão compartilhada. Esse comportamento padrão assume que os valores em camadas têm as mesmas unidades. No entanto, isso não se sustenta para este exemplo, pois estamos combinando valores de temperatura (grau Celsius) com valores de precipitação (polegadas)!

Se quisermos utilizar diferentes escalas do eixo y, precisamos de especificar como queremos que o Altair *resolva* os dados entre camadas. Neste caso, queremos resolver os domínios `escala` do eixo y para que sejam `independentes` ao invés de utilizar um domínio `compartilhado`. O objeto `Chart` produzido por um operador de camada inclui um método `resolve_scale` com o qual podemos especificar a resolução desejada:

In [30]:
tempMinMax = alt.Chart(clima).transform_filter(
  'datum.location == "Seattle"'
).mark_area(opacity=0.3).encode(
  alt.X('month(date):T', title=None, axis=alt.Axis(format='%b')),
  alt.Y('average(temp_max):Q', title='Temperatura Média °C'),
  alt.Y2('average(temp_min):Q')
)

precip = alt.Chart(clima).transform_filter(
  'datum.location == "Seattle"'
).mark_line(
  interpolate='monotone',
  stroke='grey'
).encode(
  alt.X('month(date):T'),
  alt.Y('average(precipitation):Q', title='Precipitação')
)

alt.layer(tempMinMax, precip).resolve_scale(y='independent')

alt.LayerChart(...)

_Podemos agora ver que o outono é a estação mais chuvosa em Seattle (com um pico em novembro), complementada por Verões secos._

Poderá ter notado alguma redundância nas especificações do nosso gráfico acima: ambos utilizam o mesmo conjunto de dados e o mesmo filtro para analisar apenas Seattle. Se quiser, você pode simplificar um pouco o código fornecendo os dados e a transformação do filtro para o gráfico em camadas de nível superior. As camadas individuais herdarão então os dados se não tiverem as suas próprias definições de dados:

In [31]:
tempMinMax = alt.Chart().mark_area(opacity=0.3).encode(
  alt.X('month(date):T', title=None, axis=alt.Axis(format='%b')),
  alt.Y('average(temp_max):Q', title='Temperatura Média °C'),
  alt.Y2('average(temp_min):Q')
)

precip = alt.Chart().mark_line(
  interpolate='monotone',
  stroke='grey'
).encode(
  alt.X('month(date):T'),
  alt.Y('average(precipitation):Q', title='Precipitação')
)

alt.layer(tempMinMax, precip, data=clima).transform_filter(
  'datum.location == "Seattle"'
).resolve_scale(y='independent')

alt.LayerChart(...)

Embora os gráficos de eixo duplo possam ser úteis, são frequentemente propensos de serem mal interpretados, uma vez que as diferentes unidades e escalas dos eixos podem ser enormes. Se for viável, pode considerar transformações que mapeiem diferentes campos de dados para unidades partilhadas, por exemplo, mostrando [quantis](https://en.wikipedia.org/wiki/Quantile) ou alterações percentuais relativas.